<a href="https://colab.research.google.com/github/Tokisaki-Galaxy/PterygiumSeg/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 导入必要的库
导入PyTorch、OpenCV、Pandas等必要的库，为图像分类模型做准备。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import pandas as pd
import numpy as np
import os
from PIL import Image
import platform
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.font_manager
%matplotlib inline

if platform.system() == "Windows":
    num_workers = 0
    print(f"检测到 Windows 系统，将 DataLoader 的 num_workers 设置为 {num_workers}。")
else:
    # 在非 Windows 系统（如 Linux/Colab）上
    num_workers = 4
    print(f"检测到非 Windows 系统 ({platform.system()})，将 DataLoader 的 num_workers 设置为 {num_workers}。")
    # 设置中文字体
    !wget -O simhei.ttf "https://www.wfonts.com/download/data/2014/06/01/simhei/chinese.simhei.ttf"
    matplotlib.font_manager.fontManager.addfont('simhei.ttf')
    matplotlib.rc('font', family='SimHei')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 数据路径
image_dir =          r"f:/train"
# colab路径
colab_zip_path = "/content/drive/My Drive/train.zip"
colab_extract_path = "/content/trains/"
# Kaggle路径
#kaggle_zip_path = "/kaggle/working/train.zip"
#kaggle_extract_path = "/kaggle/working/trains/"
kaggle_extract_path = "/kaggle/input/pterygium/train/"
kaggle_temp_path = "/kaggle/working/"

# 配置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA 可用: {torch.cuda.is_available()}")
print(f"使用的设备: {device}")

# 读取和准备数据
从train_classification_label.xlsx读取标签数据，并组织预处理后的图像数据路径。标签包括：0（健康）、1（建议观察）、2（建议手术）。

In [ ]:
import zipfile
import sys

# 如果在云端上运行，从 Google Drive 读取数据
if 'google.colab' in sys.modules or os.path.exists("/kaggle/working"):

    if 'google.colab' in sys.modules:
        print('在 Google Colab 环境中运行')
        image_dir = os.path.join(colab_extract_path,"train")
        label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        zip_path = colab_zip_path
        extract_path = colab_extract_path

        # Mount Google Drive
        from google.colab import drive
        drive.mount('/content/drive')
    else:
        print('在 Kaggle 环境中运行')
        # Kaggle 环境下的路径设置
        # image_dir = os.path.join(kaggle_extract_path,"train")
        # label_file = os.path.join(image_dir,"train_classification_label.xlsx")
        # zip_path = kaggle_zip_path
        # extract_path = kaggle_extract_path

        # if not os.path.exists(zip_path):
        #     from kaggle_secrets import UserSecretsClient
        #     user_secrets = UserSecretsClient()
        #     !gdown --id {user_secrets.get_secret("train_zip_downloadurl")}
        image_dir = os.path.join(kaggle_extract_path,"train")
        label_file = os.path.join(image_dir,"train_classification_label.xlsx")

    if not os.path.exists(label_file):
        # 解压数据
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)    
else:
    print(f'不在云端环境中运行,使用本地数据路径{image_dir}')
label_file = os.path.join(image_dir,"train_classification_label.xlsx")

# 自定义数据集类，用于读取图像和标签
class PterygiumDataset(Dataset):
    def __init__(self, label_file, image_dir, transform=None):
        """
        初始化数据集
        :param label_file: 包含图像标签的Excel文件路径
        :param image_dir: 图像文件夹路径
        :param transform: 图像变换操作
        """
        self.labels_df = pd.read_excel(label_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        """
        获取指定索引的图像和标签
        :param idx: 索引
        :return: 图像张量和对应标签
        """
        row = self.labels_df.iloc[idx]
        image_name = row['Image']
        label = row['Pterygium']
        image_folder = f"{int(image_name):04d}"
        image_path = os.path.join(self.image_dir, image_folder, f"{image_folder}.png")

        # 加载图像
        image = Image.open(image_path).convert("RGB")

        # 应用图像变换
        if self.transform:
            image = self.transform(image)

        return image, label

# 数据 Resize
这一步骤是将图像调整为224x224的大小，以适应模型输入要求。
只在Linux运行时使用，因为windows仅用与测试。

In [ ]:
from torchvision.transforms.functional import to_pil_image
target_size = (224, 224) # 目标尺寸
output_format = "PNG" # 输出格式

# --- Transformation Definition ---
# We will perform Resize on GPU. ToTensor conversion happens before moving to GPU.
# Normalization will be done *online* during training dataloading, not offline.
resize_transform = transforms.Resize(target_size, interpolation=transforms.InterpolationMode.BILINEAR, antialias=True)
# BILINEAR is a good default. antialias=True is recommended for downsampling.

# --- Processing Function ---
def resize_and_save_image(img_info, base_input_dir, base_output_dir, transform, device):
    """
    Reads an image, resizes it (potentially on GPU), and saves it.
    """
    try:
        image_name = img_info['Image']
        image_folder = f"{int(image_name):04d}"
        input_path = os.path.join(base_input_dir, image_folder, f"{image_folder}.png")
        
        # Create corresponding output subdirectory if it doesn't exist
        output_folder_path = os.path.join(base_output_dir, image_folder)
        os.makedirs(output_folder_path, exist_ok=True)
        output_path = os.path.join(output_folder_path, f"{image_folder}.{output_format.lower()}")

        # 1. Read image using PIL (CPU)
        img_pil = Image.open(input_path).convert("RGB")

        # 2. Convert PIL image to Tensor (CPU, scales to [0, 1])
        img_tensor_cpu = transforms.functional.to_tensor(img_pil) # Output: CxHxW

        # 3. Move tensor to GPU (if available)
        img_tensor_gpu = img_tensor_cpu.to(device)

        # 4. Apply Resize transform (GPU)
        resized_tensor_gpu = transform(img_tensor_gpu)

        # 5. Move resized tensor back to CPU
        resized_tensor_cpu = resized_tensor_gpu.cpu()

        # 6. Convert tensor back to PIL Image (CPU)
        # to_pil_image expects CxHxW tensor in [0, 1] range
        resized_img_pil = to_pil_image(resized_tensor_cpu)

        # 7. Save the resized PIL image (CPU)
        resized_img_pil.save(output_path, format=output_format)
        
        return True # Indicate success

    except FileNotFoundError:
        print(f"错误: 文件未找到 {input_path}")
        return False
    except Exception as e:
        print(f"错误处理图像 {input_path}: {e}")
        return False

if not platform.system() == "Windows":
    if 'google.colab' in sys.modules:
        original_image_dir = os.path.join(colab_extract_path,"train")
        output_dir = os.path.join(colab_extract_path,"train_resized")
    elif os.path.exists("/kaggle/working"):
        original_image_dir = os.path.join(kaggle_extract_path,"train")
        output_dir = os.path.join(kaggle_temp_path,"train_resized")
    else:
        print("错误: 无法识别的环境")
        exit(1)
    image_dir = output_dir

    print(f"输入目录: {original_image_dir}")
    print(f"输出目录: {output_dir}")
    print(f"目标尺寸: {target_size}")

    # Create the main output directory
    os.makedirs(output_dir, exist_ok=True)

    # Read label file to know which images to process
    try:
        labels_df = pd.read_excel(label_file)
    except FileNotFoundError:
        print(f"错误: 标签文件未找到 {label_file}")
        sys.exit(1) # Exit if label file is missing

    success_count = 0
    error_count = 0

    # Iterate through images listed in the label file
    for index, row in tqdm(labels_df.iterrows(), total=len(labels_df), desc="Resizing Images"):
        if resize_and_save_image(row, original_image_dir, output_dir, resize_transform, device):
            success_count += 1
        else:
            error_count += 1

    print(f"\n处理完成!")
    print(f"成功处理图像数: {success_count}")
    print(f"处理失败图像数: {error_count}")
    print(f"处理后的图像保存在: {output_dir}")

# 创建数据加载器
使用PyTorch的Dataset和DataLoader类创建数据集和加载器，包括数据增强和训练/验证集的划分。

In [ ]:
# 数据变换
train_transform = transforms.Compose([
    transforms.Resize((256, 256)), # 先放大一点
    transforms.RandomCrop((224, 224)), # 随机裁剪回目标尺寸
    transforms.RandomHorizontalFlip(p=0.5), # 随机水平翻转
    transforms.RandomRotation(degrees=15), # 随机旋转
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), # 随机颜色抖动
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 定义验证集/测试集的变换 (无需数据增强)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 划分训练集和验证集，并创建对应的数据加载器
from sklearn.model_selection import train_test_split

# 读取标签文件
labels_df = pd.read_excel(label_file)

# 按照8:2的比例划分训练集和验证集
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['Pterygium'])

# 保存划分后的数据集到文件
train_label_file = os.path.join(image_dir, "train_classification_label_train.xlsx")
val_label_file = os.path.join(image_dir, "train_classification_label_val.xlsx")
if os.path.exists("/kaggle/working"):
    train_label_file = os.path.join(kaggle_temp_path, "train_classification_label_train.xlsx")
    val_label_file = os.path.join(kaggle_temp_path, "train_classification_label_val.xlsx")
train_df.to_excel(train_label_file, index=False)
val_df.to_excel(val_label_file, index=False)

# 创建训练集和验证集的数据集对象 (使用不同的 transform)
train_dataset = PterygiumDataset(label_file=train_label_file, image_dir=image_dir, transform=train_transform) # 使用训练变换
val_dataset = PterygiumDataset(label_file=val_label_file, image_dir=image_dir, transform=val_transform) # 使用验证变换

# 创建训练集和验证集的数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=num_workers, pin_memory=False if platform.system() == "Windows" else True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=False if platform.system() == "Windows" else True)

# 构建 ResNet18 模型
使用PyTorch的预训练ResNet18模型，修改最后的全连接层以适应3个类别的分类任务。

In [ ]:
# 构建 ResNet18 模型
from torchvision.models import ResNet18_Weights
class ResNet18Classifier(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet18Classifier, self).__init__()
        # 加载预训练的 ResNet18 模型
        self.resnet18 = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        # 替换最后的全连接层以适应3个类别的分类任务
        in_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet18(x)

# 定义模型
model = ResNet18Classifier(num_classes=3)

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 将模型移动到 GPU（如果可用）
model = model.to(device)

# 定义带正则化项的损失函数
实现一个包含正则化项的损失函数，使用交叉熵损失作为基础，并添加特定的正则化项来抑制高光问题。

In [ ]:
# 定义损失函数，包含正则化项以抑制高光问题
class HighlightRegularizedLoss(nn.Module):
    def __init__(self, base_loss_fn, lambda_reg=0.01):
        super(HighlightRegularizedLoss, self).__init__()
        self.base_loss_fn = base_loss_fn
        self.lambda_reg = lambda_reg

    def forward(self, outputs, targets, inputs):
        # 基础损失（交叉熵损失）
        base_loss = self.base_loss_fn(outputs, targets)

        # 正则化项：抑制高光问题（假设高光区域的像素值接近1）
        highlight_penalty = torch.mean(torch.clamp(inputs - 0.9, min=0) ** 2)

        # 总损失
        total_loss = base_loss #+ self.lambda_reg * highlight_penalty
        return total_loss

# 定义基础损失函数（交叉熵损失）
base_loss_fn = nn.CrossEntropyLoss()

# 定义包含正则化项的损失函数
criterion = HighlightRegularizedLoss(base_loss_fn=base_loss_fn, lambda_reg=0.01)

# 配置优化器和训练参数
设置Adam或SGD优化器，学习率调度器和其他训练参数，为模型训练做准备。

In [ ]:
# 配置优化器和学习率调度器
# 在Adam优化器中添加 weight_decay 参数实现L2正则化
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# 定义学习率调度器，采用余弦退火调度策略
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

# 设置其他训练参数
num_epochs = 25  # 训练的总轮数
log_interval = 10  # 每隔多少个批次打印一次日志

# 训练模型
实现训练循环，包括前向传播、损失计算、反向传播和参数更新，并记录训练过程中的指标。

In [ ]:
# 定义早停类
from copy import deepcopy
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.0, mode='max'):
        self.patience = patience
        self.min_delta = min_delta # 允许的最小提升量
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.mode = mode
        self.best_model_weights = None
        
        # 根据模式确定比较操作
        if self.mode == 'min':
            self.delta_sign = -1 # 对于最小值模式，分数需要减少 delta
        else: # mode == 'max'
            self.delta_sign = 1 # 对于最大值模式，分数需要增加 delta
            
    def __call__(self, val_score, model):
        score = val_score # 直接使用验证分数
        
        if self.best_score is None:
            # 第一次调用，初始化最佳分数并保存权重
            self.best_score = score
            self.best_model_weights = deepcopy(model.state_dict())
            print(f"EarlyStopping: Initialized best score to {self.best_score:.4f}")
        # 检查是否有足够的提升
        elif (score * self.delta_sign) > (self.best_score * self.delta_sign) + self.min_delta:
            # 有足够的提升
            self.best_score = score
            self.best_model_weights = deepcopy(model.state_dict())
            self.counter = 0
            print(f"EarlyStopping: Improvement found. Best score updated to {self.best_score:.4f}. Counter reset.")
        else:
            # 没有足够的提升
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}. Best score remains {self.best_score:.4f}.')
            if self.counter >= self.patience:
                self.early_stop = True
                print("EarlyStopping: Patience reached.")

In [ ]:
# 训练模型
# 初始化 GradScaler
scaler = torch.amp.GradScaler('cuda')
# 初始化早停
early_stopping = EarlyStopping(patience=5, mode='max')  # 使用验证准确率

for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    train_loss = 0.0
    correct = 0
    total = 0
    
    # 使用tqdm创建进度条
    train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)  # 将数据移动到 GPU（如果可用）

        # 训练循环中修改前向传播和反向传播部分
        optimizer.zero_grad()  # 清空梯度

        # 前向传播和损失计算使用混合精度
        with torch.amp.autocast('cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, targets, inputs)

        # 使用scaler进行反向传播
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # 记录损失和准确率
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # 更新进度条信息
        current_lr = optimizer.param_groups[0]['lr'] # 获取当前学习率
        train_loader_tqdm.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100. * correct / total:.2f}%',
            'lr': f'{current_lr:.1e}'
        })
        
        # 打印训练日志
        if (batch_idx + 1) % log_interval == 0:
            tqdm.write(f"Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], "
                  f"Loss: {loss.item():.4f}, Accuracy: {100. * correct / total:.2f}%")

    # 学习率调度器更新
    scheduler.step()

    # 验证模型
    model.eval()  # 设置模型为评估模式
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # 前向传播和计算损失
            with torch.amp.autocast('cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, targets, inputs)
            val_loss += loss.item()

            # 记录准确率
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()

    # 打印验证结果
    tqdm.write(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {100. * correct / total:.2f}%, Val Loss: {val_loss / len(val_loader):.4f}, "
          f"Val Accuracy: {100. * val_correct / val_total:.2f}%")
    
    # 早停检测
    val_accuracy = 100. * val_correct / val_total
    early_stopping(val_accuracy, model)
    
    # 判断是否需要早停
    if early_stopping.early_stop:
        print("早停触发！在验证集上的表现不再提升。")
        # 恢复最佳模型权重
        model.load_state_dict(early_stopping.best_model_weights)
        break

# 评估模型性能
在验证集上评估模型性能，计算准确率、混淆矩阵、F1分数等指标，并可视化结果。

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import seaborn as sns

# 评估模型性能
model.eval()  # 设置模型为评估模式
all_targets = []
all_predictions = []

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # 前向传播
        outputs = model(inputs)

        # 获取预测结果
        _, predicted = outputs.max(1)
        all_targets.extend(targets.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# 计算评估指标
accuracy = accuracy_score(all_targets, all_predictions)
f1 = f1_score(all_targets, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_targets, all_predictions)

print(f"验证集准确率: {accuracy:.4f}")
print(f"验证集F1分数: {f1:.4f}")

# 可视化混淆矩阵
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["健康", "建议观察", "建议手术"],
            yticklabels=["健康", "建议观察", "建议手术"])
plt.xlabel("预测标签")
plt.ylabel("真实标签")
plt.title("混淆矩阵")
plt.show()

# 模型测试和预测
使用训练好的模型对新图像进行预测，并展示几个预测示例。

In [ ]:
# 模型测试和预测
def predict_image(model, image_path, transform, device):
    """
    使用训练好的模型对单张图像进行预测
    :param model: 训练好的模型
    :param image_path: 图像路径
    :param transform: 图像预处理变换
    :param device: 设备（CPU 或 GPU）
    :return: 预测类别
    """
    model.eval()  # 设置模型为评估模式
    image = Image.open(image_path).convert("RGB")  # 加载图像并转换为RGB
    image = transform(image).unsqueeze(0).to(device)  # 应用预处理并添加批次维度

    with torch.no_grad():
        outputs = model(image)  # 前向传播
        _, predicted = outputs.max(1)  # 获取预测类别
    return predicted.item()

# 示例预测
test_image_dir = "./test_images"  # 测试图像文件夹路径
test_images = os.listdir(test_image_dir)[:5]  # 获取测试图像文件夹中的前5张图像

# 对每张测试图像进行预测并展示结果
for image_name in test_images:
    image_path = os.path.join(test_image_dir, image_name)
    predicted_class = predict_image(model, image_path, transform, device)
    class_names = ["健康", "建议观察", "建议手术"]
    print(f"图像: {image_name}, 预测类别: {class_names[predicted_class]}")

    # 可视化图像及其预测结果
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(f"预测类别: {class_names[predicted_class]}")
    plt.axis("off")
    plt.show()

# 模型保存和加载
保存训练好的模型参数，以便将来使用，并展示如何加载保存的模型进行推理。

In [ ]:
# 保存模型参数
def save_model(model, path):
    """
    保存模型参数到指定路径
    :param model: 训练好的模型
    :param path: 保存路径
    """
    torch.save(model.state_dict(), path)
    print(f"模型参数已保存到 {path}")

# 加载模型参数
def load_model(model, path, device):
    """
    从指定路径加载模型参数
    :param model: 模型实例
    :param path: 模型参数路径
    :param device: 设备（CPU 或 GPU）
    :return: 加载参数后的模型
    """
    model.load_state_dict(torch.load(path, map_location=device))
    model = model.to(device)
    print(f"模型参数已从 {path} 加载")
    return model

# 示例：保存训练好的模型
model_save_path = "./resnet18_pterygium_classifier.pth"
save_model(model, model_save_path)

# 示例：加载保存的模型并进行推理
loaded_model = ResNet18Classifier(num_classes=3)
loaded_model = load_model(loaded_model, model_save_path, device)

# 测试加载的模型是否能够正常推理
test_image_path = "./test_images/sample_image.png"  # 替换为实际测试图像路径
predicted_class = predict_image(loaded_model, test_image_path, transform, device)
class_names = ["健康", "建议观察", "建议手术"]
print(f"加载模型后预测结果: 图像 {os.path.basename(test_image_path)}, 预测类别: {class_names[predicted_class]}")